In [7]:
#%pip install yfinance pandas requests beautifulSoup4
import yfinance as yf
import pandas as pd
import requests
import os
import json
import time
from bs4 import BeautifulSoup

In [ ]:
def get_financial_statement(sheet_type,co_id, industry, year, season, base_dir):


    """ 
    sheet_type 會計表類型 = I(income statement) B(Balance sheet) S(Statement of cash flows)
    co_id="" #四位數公司代碼
    year="" #國歷年分
    season="" #1~4 
    """
    sheet_code=""
    if (sheet_type == "I"):
        sheet_code = "4"
    elif (sheet_type == "B"):
        sheet_code = "3"
    elif (sheet_type == "S"):
        sheet_code = "5"
    else:
        print("Invalid sheet_type!")
        return None


    url = "https://mopsov.twse.com.tw/mops/web/t164sb0{sheet_code}?encodeURIComponent=1&step=1&firstin=1&off=1&keyword4=&code1=&TYPEK2=&checkbtn=&queryName=co_id&inpuType=co_id&TYPEK=all&isnew=false&co_id={co_id}&year={year}&season=0{season}"
    headers = {"User-Agent": "Mozilla/5.0"}

    # 取得網頁內容
    response = requests.get(url, headers=headers)
    response.encoding = 'utf8'  # 根據網頁編碼設定，若非 Big5 可調整
    html_content = response.text


    # 使用 BeautifulSoup 解析 HTML
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
    except ValueError :
        print(f"response has nothing to do. co_id:{co_id}")

    # 找到 <div id="table01">
    div_table = soup.find('div', id='table01')
    if div_table:
        # 將 <div id="table01"> 中的 HTML 轉換為字串，再用 pandas 讀取表格
        table_html = str(div_table)
        tables = pd.read_html(table_html)
        save_path = os.path.join(base_dir,industry,year,season)
        if not os.path.exists(base_dir):
            os.makedirs(base_dir)
        tables[1].to_csv(os.path.join(save_path,"TW_{co_id}_{year}_{season}_{sheet_type}.csv"), index=False, encoding="utf-8-sig") # 第一個table1是目錄 需要的是第二個
        print(tables)
    else : 
        print("爬蟲失敗 沒有找到表格")

    print(f"TW_{co_id}_{year}_{season}_{sheet_type}.csv爬蟲完成,休眠5秒")
    time.sleep(5) # 處理防爬 不確定秒數夠不夠

In [14]:
# 讀取 JSON 檔案
json_path = "TW_stock.json"
with open(json_path, "r", encoding="utf-8") as f:
    tw_stock = json.load(f)

# 檔案儲存根目錄
base_dir = "./TW"

# 確保目錄存在
if not os.path.exists(base_dir):
    os.makedirs(base_dir)


end_year = 114
year_range = 10

for company_code, info in tw_stock.items():
    code = info.get("代號")
    industry = info.get("產業別")

    if(len(code) == 4 and industry != ""): #過濾非公司股票 
        print(code,"  ",industry)
        for year in range (end_year-year_range,end_year):
            for season in range(1,5):
                get_financial_statement("I", code, industry, year, season, base_dir) # 取得損益表
                get_financial_statement("B", code, industry, year, season, base_dir) # 取得資產負債表
                get_financial_statement("S", code, industry, year, season, base_dir) # 取得現金流量表
        


    


'''

co_id="" #四位數公司代碼
year="" #國歷年分
season="" #1~4
url = "https://mopsov.twse.com.tw/mops/web/t164sb03?encodeURIComponent=1&step=1&firstin=1&off=1&keyword4=&code1=&TYPEK2=&checkbtn=&queryName=co_id&inpuType=co_id&TYPEK=all&isnew=false&co_id={co_id}&year={year}&season=0{season}"
headers = {"User-Agent": "Mozilla/5.0"}

# 取得網頁內容
response = requests.get(url, headers=headers)


response.encoding = 'utf8'  # 根據網頁編碼設定，若非 Big5 可調整
html_content = response.text

# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(html_content, 'html.parser')

# 找到 <div id="table01">
div_table = soup.find('div', id='table01')
if div_table:
    # 將 <div id="table01"> 中的 HTML 轉換為字串，再用 pandas 讀取表格
    table_html = str(div_table)
    tables = pd.read_html(table_html)
    tables[1].to_csv("table01.csv", index=False, encoding="utf-8-sig")
    print(tables)

'''

1101    水泥工業
TW_1101_104_1_I.csv爬蟲完成,休眠5秒
TW_1101_104_1_B.csv爬蟲完成,休眠5秒
TW_1101_104_1_S.csv爬蟲完成,休眠5秒
TW_1101_104_2_I.csv爬蟲完成,休眠5秒


KeyboardInterrupt: 

In [ ]:

def financial_statement(year, season, report_type='綜合損益彙總表'):
    """
    抓取公開資訊觀測站中指定年度、季別的財報資料
    report_type 可設定為：
      '綜合損益彙總表'  → 損益表
      '資產負債彙總表'  → 資產負債表
      '營益分析彙總表'  → 營利分析表
    注意：若輸入西元年，程式內部會先轉為民國年。
    """
    # 若傳入西元年，轉為民國年
    if year > 1000:
        year -= 1911

    # 根據報表類型選擇對應的 URL
    if report_type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif report_type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif report_type == '營益分析彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        raise ValueError("不支援的報表類型！請選 '綜合損益彙總表'、'資產負債彙總表' 或 '營益分析彙總表'。")
    
    # 設定 POST 請求的參數
    payload = {
        'encodeURIComponent': 1,
        'step': 1,
        'firstin': 1,
        'off': 1,
        'keyword4':"", 
        'code1': "",
        'TYPEK2':"" ,
        'checkbtn':"", 
        'queryName': 'co_id',
        'inpuType': 'co_id',
        'TYPEK': all,
        'isnew': False,
        'co_id': "1101",
        'year': "110",
        'season': "02"
    }
    
    #url = "https://mopsov.twse.com.tw/mops/web/t164sb03?encodeURIComponent=1&step=1&firstin=1&off=1&keyword4=&code1=&TYPEK2=&checkbtn=&queryName=co_id&inpuType=co_id&TYPEK=all&isnew=false&co_id=1101&year=110&season=02"
    url = "https://mopsov.twse.com.tw/server-java/t164sb01?step=1&CO_ID=1101&SYEAR=2021&SSEASON=2&REPORT_ID=C#BalanceSheet"
    # 發出 POST 請求
    r = requests.post(url)
    # 預設編碼可能正確，也可以試試 utf8 或 big5，這裡通常使用 utf8
    r.encoding = 'big5'
    
    # 利用 pandas 讀取 HTML 中的表格
    try:
        dfs = pd.read_html(r.text, header=None)
    except ValueError:
        raise RuntimeError("無法從回傳資料中解析出表格，請檢查是否因查詢條件（例如當季財報尚未揭露）而無資料。")
    
    '''
    # 假設第一個表格通常為標題說明，取後續的所有表格並合併
    df = pd.concat(dfs[1:], axis=0, sort=False)
    # 若表格內有多層欄位，可嘗試用以下方式調整：
    if hasattr(df.columns, 'levels'):
        df.columns = df.columns.get_level_values(1)
    
    # 以下範例示意轉換數值，實際情況可能需要依據報表格式做進一步處理
    df = df.apply(lambda col: pd.to_numeric(col, errors='coerce'))
    '''
    for i, table in enumerate(dfs):

        print(f"表格 {i}：")
        print(table.head())
        print("-" * 50)
        if ( "資產負債表Balance Sheet" in table.to_string()):
            table.head().to_csv(f"TW/test/{i}.csv",index = False)
    # TODO: 分成每個公司的三表 #########################

    """
    將單一公司爬下來的三表資料依照以下規則儲存：
      1. base_dir (例如 "TW")
      2. 產業分類 (例如 "電子業")
      3. 公司資料夾 (以 TW 股票代碼命名，如 "2330")
      4. 各個報表資料（儲存為 CSV 檔）
      
    同時刪除每個 DataFrame 中所有完全為空值的欄位與列。
    
    """

    # 讀取 JSON 檔案
    json_path = "/mnt/data/TW_stock.json"
    with open(json_path, "r", encoding="utf-8") as f:
        stock_data = json.load(f)

    # 檔案儲存根目錄
    base_dir = "TW"

    # 確保目錄存在
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # 遍歷 CSV 每一行
    for _, row in df.iterrows():
        stock_code = str(row["代號"])  # 假設 CSV 中有 "代號" 欄位

        # 獲取產業分類
        industry = stock_data.get(stock_code, {}).get("產業別", "未知分類")

        # 建立目錄 `TW/產業分類/代碼/`
        save_path = os.path.join(base_dir, industry, stock_code, year)
        os.makedirs(save_path, exist_ok=True)

        # 儲存該行 CSV
        row_df = pd.DataFrame([row])
        row_df=row_df.dropna(axis=1, how='all').dropna(axis=0, how='all')
        row_df.to_csv(os.path.join(save_path, f"{stock_code}.csv"), index=False, encoding='utf8')
        print(f"已儲存：{save_path}")


    


    return True


################################################################################

# 使用範例：抓取 2020 年第一季（民國 109 年第一季）的綜合損益彙總表（損益表）
if __name__ == "__main__":
    print(financial_statement(109, 1, report_type='綜合損益彙總表'))
    
    
    


In [ ]:

def save_company_reports(base_dir, company_data):
    """
    將單一公司爬下來的三表資料依照以下規則儲存：
      1. base_dir (例如 "TW")
      2. 產業分類 (例如 "電子業")
      3. 公司資料夾 (以 TW 股票代碼命名，如 "2330")
      4. 各個報表資料（儲存為 CSV 檔）
      
    同時刪除每個 DataFrame 中所有完全為空值的欄位與列。
    
    """
    stock_code = company_data.get("公司代號")
    industry = company_data.get("產業分類")
    
    # 報表的鍵，可以根據你的資料調整
    report_keys = ["綜合損益彙總表", "資產負債彙總表", "營益分析彙總表"]
    
    # 建立資料夾層級：base_dir/產業分類/股票代碼
    company_dir = os.path.join(base_dir, str(industry), str(stock_code))
    os.makedirs(company_dir, exist_ok=True)
    
    for key in report_keys:
        df = company_data.get(key)
        if df is None:
            print(f"{key} 資料不存在，跳過。")
            continue
        # 刪除所有全為 NaN 的欄位與列
        df_clean = df.dropna(axis=1, how='all').dropna(axis=0, how='all')
        # 儲存為 CSV（可根據需要調整編碼）
        file_path = os.path.join(company_dir, f"{key}.csv")
        df_clean.to_csv(file_path, index=False, encoding='utf8')
        print(f"已儲存：{file_path}")

# 範例：假設你已爬到一家公司的資料
if __name__ == "__main__":
    
    company_data = financial_statement(109, 1, report_type='綜合損益彙總表')
    base_directory = "./TW"  # 最外層資料夾
    save_company_reports(base_directory, company_data)